In [1]:
import numpy as np
import cPickle
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics.pairwise import pairwise_distances
import itertools
import string
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import *
#import pyemd
#loading word embeddings
#wv = KeyedVectors.load_word2vec_format("../w2v/w2vemb.bin", binary=True)

# Visualize functions

In [2]:
def write_file(n_clusters_, labels, corpus, mapping_dict, test_corpus):
    for indice_cluster in range(n_clusters_):
        # print indice_cluster
        idx_list = np.where(labels == indice_cluster)[0]
        f1 = open('cluster_%s/%dth_cluster.txt' % (question_class, indice_cluster), 'w+')
        for idx in idx_list:
            f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
    f1.close()


def write_ne(nes, question_class, mapping_dict):
    f1 = open('ne_%s.txt' % question_class, 'w+')
    for idx, doc in enumerate(nes):
        f1.write('%s %s\n' % (mapping_dict[idx], ' '.join(doc)))
    f1.close()    

def wmd_compute(x1, x2):
    return wv.wmdistance(x1, x2)

def analyze_cluster(n_clusters_, labels, corpus, mapping_dict, test_corpus, filename):
    num = 0
    other_content = []
    for indice_cluster in range(n_clusters_):
        idx_list = np.where(labels == indice_cluster)[0]
        if len(idx_list) > 4:
            f1 = open('%s_%s/%dth_cluster.txt' % (filename, question_class, indice_cluster), 'w+')
            for idx in idx_list:
                f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
            f1.close()
        if len(idx_list) < 4:
            num = num + len(idx_list)
            for idx in idx_list:
                other_content.append(corpus[mapping_dict[idx] - 1])
    f1 = open('%s_%s/other_cluster.txt' % (filename, question_class), 'w+')
    for doc in other_content:
        f1.write('%s\n' % doc)
    f1.close()
    print num

# Reading Raw text

In [3]:
question_class = 'q1'
fname = '..//data//%s.txt' % question_class
with open(fname) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]
print 'length of content', len(content)
f.close()
print content

length of content 168
['after taking appointment then also i have to wait 1 to 1 and half hour . even there was no rush , work should be done immediately .', 'service center should provide comfortable time for servicing from there end . or they should arrange service like we can do booking through messages . also toyota has given only one number for all services .', 'after arranging the appointment they should quickly service the car and also delivery of car should fast .', 'they pick up the phone and then disconnect it . they tell their schedule is busy , they say it will take some time we will call you but they never call back . and when we call the service station then immediately they ask us to send the car . they should tell us the schedule properly .', 'toyota have an app , it \\? s a mobile app which is not applicable for all the customers . first of all , this app should be for every customer . this app should be available for all the users . also this app should be free for al

In [4]:
# split comment with multi-sentence into multi-comments
content_new = []
for comment in content:
    sents = sent_tokenize(comment)
    if len(sents) > 1:
        for i in range(len(sents)):
            content_new.append(sents[i])
    else:
        content_new.append(comment)
print 'length of new content', len(content_new)
print content_new
    

length of new content 395
['after taking appointment then also i have to wait 1 to 1 and half hour .', 'even there was no rush , work should be done immediately .', 'service center should provide comfortable time for servicing from there end .', 'or they should arrange service like we can do booking through messages .', 'also toyota has given only one number for all services .', 'after arranging the appointment they should quickly service the car and also delivery of car should fast .', 'they pick up the phone and then disconnect it .', 'they tell their schedule is busy , they say it will take some time we will call you but they never call back .', 'and when we call the service station then immediately they ask us to send the car .', 'they should tell us the schedule properly .', 'toyota have an app , it \\?', 's a mobile app which is not applicable for all the customers .', 'first of all , this app should be for every customer .', 'this app should be available for all the users .', 'a

In [5]:
stemmer = PorterStemmer()
print stemmer.stem("location")

locat


In [6]:

def rule_q1(sen, ne):
    clean_ne = list(set(ne))
    # remove_words = ['customer', 'customers', 'waiting', 'facility', 'facilities', 'dealership', 'toyota', 'improvement']
    remove_words = ["appointment","appointments", "improve","improved", "customer", "customers",\
                    "service","servicing","services", "people","person","persons","facility","facilities",\
                    "availible","good","center","centers","centre","car","cars", "dealership", "dealerships",\
                    "vehicle","vehicles", "toyota", "problem","problems","work","works", "much",\
                   "thing","things", "possible","need","needs"]   
    clean_ne = [word for word in clean_ne if word not in remove_words]
    
    save_words = ["without", "call","called","wait","waiting", "pick", "picked","pickup", "drop",\
                  "remind","reminded", "inform","informed", "response", "book","booked","booking","fix","fixed","fixing",\
                 "receive", "receiving","received","same", "sms", "immediate", "immediately",\
                  "urgent","deliver","delivered","explain","explaining","explained", "detail","details"]
    clean_ne = clean_ne + [word for word in sen.split() if word in save_words]
    
    # rules to merge keywords:
    if 'pickup' in clean_ne:
        clean_ne[clean_ne.index('pickup')] = 'pick'
    if 'phone' in clean_ne:
        clean_ne[clean_ne.index('phone')] = 'call'
    if 'telephone' in clean_ne:
        clean_ne[clean_ne.index('telephone')] = 'call'
    if 'message' in clean_ne:
        clean_ne[clean_ne.index('message')] = 'sms'
    if 'messages' in clean_ne:
        clean_ne[clean_ne.index('messages')] = 'sms'
    if 'km' in clean_ne:
        clean_ne[clean_ne.index('km')] = 'location'
    if 'area' in clean_ne:
        clean_ne[clean_ne.index('area')] = 'location'      
    if 'urgent' in clean_ne:
        clean_ne[clean_ne.index('urgent')] = 'emergency'
    if 'detail' in clean_ne:
        clean_ne[clean_ne.index('detail')] = 'inform'
    if 'details' in clean_ne:
        clean_ne[clean_ne.index('details')] = 'inform'
    if 'explain' in clean_ne:
        clean_ne[clean_ne.index('explain')] = 'inform'
    if 'explaining' in clean_ne:
        clean_ne[clean_ne.index('explaining')] = 'inform'
    if 'should tell' in sen:
        clean_ne.append('inform')
    # add keyword "appointment"
    if 'book' in clean_ne:
        clean_ne[clean_ne.index('book')] = 'appointment'
    if 'booking' in clean_ne:
        clean_ne[clean_ne.index('booking')] = 'appointment'
    if 'day' in clean_ne:  # split 'day' to 'appointment' and 'wait'
        if 'appointment' in sen:
            clean_ne[clean_ne.index('day')] = 'appointment'
        else:
            clean_ne.remove('day')
    if 'days' in clean_ne:  # split 'days' to 'appointment' and 'wait'
        if 'appointment' in sen:
            clean_ne[clean_ne.index('days')] = 'appointment'
        else:
            clean_ne[clean_ne.index('days')] = 'wait'
    if 'week' in clean_ne and 'appointment' in sen:
        clean_ne[clean_ne.index('week')] = 'appointment'
    if 'weeks' in clean_ne and 'appointment' in sen:
        clean_ne[clean_ne.index('weeks')] = 'appointment'
    if 'month' in clean_ne and 'appointment' in sen:
        clean_ne[clean_ne.index('month')] = 'appointment'
    if 'months' in clean_ne and 'appointment' in sen:
        clean_ne[clean_ne.index('months')] = 'appointment'
    # add keyword "without" appointment
    if 'without appointment' in sen or 'without any appointment' in sen:
        clean_ne.append('without')
    # add keyword "punctual" (on time)
    if 'on time' in sen:
        clean_ne.append('punctual')
    if 'that time' in sen and ('servic' in sen or 'work' in sen or 'attend' in sen):
        clean_ne.append('punctual')
    if 'fulfill' in sen or 'stick' in sen:
        clean_ne.append('punctual')
    # split "immediately" into "wiat" and "appointment" cluster
    if 'immediately' in clean_ne:
        if 'servic' in sen or 'work' in sen or 'attend' in sen:
            clean_ne[clean_ne.index('immediately')] = 'wait'
        else:
            clean_ne[clean_ne.index('immediately')] = 'appointment'
    # split 'time' to other clusters
    if 'too much time' in sen:
        clean_ne[clean_ne.index('time')] = 'wait'
    if 'long time' in sen:
        if 'appointment' in sen:
            clean_ne[clean_ne.index('time')] = 'appointment'
        else:
            clean_ne[clean_ne.index('time')] = 'wait'
    if 'time' in clean_ne:
        if 'appointment' in sen:
            clean_ne[clean_ne.index('time')] = 'appointment'
        else:
            clean_ne.remove('time')   # remove other 'times'
        
    clean_ne = list(set(clean_ne))
    return clean_ne

def clean_corpus(x1):
    x_new = []
    for x in x1:
        if 'no improvement' in x:
            continue
        else:
            x_new.append(x)
    return x_new

def process_corpus(x1, pos_tags, general_stop):
    # split three categories: 1 no improvemnt 2 with noun 3 others
    doc_noimprove = []
    doc_nn = []
    nn_extracted = []
    doc_other = []
    for x in x1:
        if 'no improvement' in x:
            doc_noimprove.append(x)
        else:
                nn_list = []
                sen = x
                pos_new = nltk.pos_tag(nltk.word_tokenize(sen))
                for token in pos_new:
                    if token[1] in pos_tags and not token[0] in general_stop:
                        nn_list.append(token[0])
#                 #stemming
#                 for counter,word in enumerate(nn_list):
#                     nn_list[counter] = stemmer.stem(word)
                # apply rule
                nn_list = rule_q1(sen, nn_list)
                    
                if nn_list != []:
                    nn_extracted.append(nn_list)
                    doc_nn.append(sen)
                else:
                    doc_other.append(sen)
    return doc_noimprove, [doc_nn, nn_extracted], doc_other
        
nn_corpus = []

# English stop words lists
stop_words = stopwords.words('english')
punctuation_list = [unicode(i) for i in string.punctuation]
for punctuation in punctuation_list:
    stop_words.append(punctuation)

pos_tags = ['NN', 'NNS','JJ','JJR','JJS']

doc1, doc2, doc3 = process_corpus(content_new, pos_tags, stop_words)

doc_nn, nn_extracted = doc2[0], doc2[1]
print 'signle sentence with nn', len(doc_nn)
print 'no comments', len(doc1)
print 'single sentence without nn', len(doc3)
print nn_extracted

signle sentence with nn 326
no comments 9
single sentence without nn 60
[['hour', 'wait'], ['rush', 'wait'], ['end', 'comfortable'], ['appointment', 'sms'], ['number'], ['delivery'], ['call', 'pick'], ['busy', 'call', 'schedule'], ['station', 'call', 'wait'], ['inform', 'schedule'], ['app'], ['mobile', 'app', 'applicable'], ['app'], ['available', 'app', 'users'], ['app', 'users', 'free'], ['app', 'fix'], ['advice', 'expert'], ['information', 'proactive'], ['driven', 'kilometer'], ['mobile', 'fixing', 'app'], ['care'], ['appointment', 'last'], ['owner', 'consult'], ['appointment', 'punctual', 'unnecessary', 'take', 'wait', 'staff'], ['market'], ['company', 'renold', 'manner', 'quick', 'response', 'treat'], ['jd', 'location'], ['showroom', 'call', 'number', 'pick'], ['city', 'showroom', 'number'], ['difficulties', 'number'], ['frame', 'punctual'], ['city', 'punctual'], ['procedure'], ['many', 'long', 'wait'], ['appointment'], ['attention'], ['fixing', 'improvement'], ['way'], ['proper', 

In [7]:
from collections import Counter
def df_count(x1):
    text = []
    for ab in x1:
        text = text + ab
    df = Counter(text)
    return df
df = df_count(nn_extracted)
print df

Counter({'appointment': 75, 'wait': 38, 'call': 37, 'punctual': 24, 'pick': 21, 'drop': 15, 'location': 13, 'available': 13, 'number': 12, 'staff': 11, 'emergency': 9, 'inform': 9, 'hour': 8, 'free': 7, 'app': 7, 'called': 7, 'same': 7, 'fix': 6, 'mobile': 6, 'improvement': 5, 'sms': 5, 'waiting': 5, 'booking': 5, 'hours': 5, 'pune': 5, 'receive': 5, 'long': 5, 'online': 5, 'month': 4, 'busy': 4, 'early': 4, 'timing': 4, 'proper': 4, 'space': 4, 'job': 4, 'care': 4, 'pickup': 4, 'response': 4, 'convenient': 4, 'half': 3, 'kilometer': 3, 'quick': 3, 'morning': 3, 'receiving': 3, 'schedule': 3, 'deliver': 3, 'last': 3, 'parts': 3, 'place': 3, 'book': 3, 'advisor': 3, 'next': 3, 'station': 3, 'fast': 3, 'give': 3, 'provide': 3, 'fixing': 3, 'without': 3, 'take': 3, 'request': 3, 'numbers': 2, 'satisfied': 2, 'outside': 2, 'easy': 2, 'local': 2, 'details': 2, 'period': 2, 'fullest': 2, 'companies': 2, 'remind': 2, 'fixed': 2, 'banaras': 2, 'crowd': 2, 'home': 2, 'charges': 2, 'lots': 2, 'e

# Compute centroid of the word vectors for each comment

In [8]:
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

filename = "/data1/shared_all/GoogleNews-vectors-negative300.bin"
word2vec_model = KeyedVectors.load_word2vec_format(filename, binary=True)
word2vec_model.init_sims(replace=True)

def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.vocab]
    return np.mean(word2vec_model[doc], axis=0)

def filter_docs(nn_extracted, doc_nn, word2vec_model):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    nn_extracted_filtered = []
    doc_nn_filtered = []
    num_filtered = 0
    for counter, doc in enumerate(nn_extracted):
        has_vector_representation = not all(word not in word2vec_model.vocab for word in doc)
        if has_vector_representation:
            nn_extracted_filtered.append(doc)
            doc_nn_filtered.append(doc_nn[counter])
        else:
            num_filtered += 1 
    print "%d doc filtered" %num_filtered
    return nn_extracted_filtered, doc_nn_filtered

nn_extracted, doc_nn = filter_docs(nn_extracted, doc_nn, word2vec_model)



similarity_matrix = cosine_similarity(np.array([document_vector(word2vec_model, doc)
                                                       for doc in nn_extracted]))


Using TensorFlow backend.


1 doc filtered


# Clustering

In [9]:
from sklearn.cluster import SpectralClustering
import os

n_clusters_ = 10
sc = SpectralClustering(n_clusters=n_clusters_, affinity='precomputed').fit(similarity_matrix)
labels = sc.labels_
print('Estimated number of clusters: %d' % n_clusters_)
print len(labels)

def write_file(n_clusters_, labels, doc_nn, nn_extracted):
    if (not os.path.isdir("cluster_%s" %question_class)):
        os.mkdir("cluster_%s" %question_class)
    for indice_cluster in range(n_clusters_):
        # print indice_cluster
        idx_list = np.where(labels == indice_cluster)[0]
        f1 = open('cluster_%s/%dth_cluster.txt' % (question_class, indice_cluster), 'w+')
        for idx in idx_list:
            f1.write('%s %s\n' % (doc_nn[idx], nn_extracted[idx]))
    f1.close()
    print("clusters written to folder: cluster_%s" %question_class)
    
write_file(n_clusters_, labels, doc_nn, nn_extracted)

Estimated number of clusters: 10
325
clusters written to folder: cluster_q1
